# Run this first cell to define all the functions:

In [2]:
import financialmath as FM

# 2 securities (K1 and K2), 3 scenarios ==> find everthing else

In [4]:
p = ["0.2", "0.4", "0.4"]
k1 = ["-0.1", "0","0.1"]
k2 = ["0.05", "0.1", "-0.05"]

FM.two_securities(p, k1, k2)


feasable set without short selling, -1 < p12 < sigma1/sigma2


,Variable,Formula,Value
0,E[K1],E[K1],0.02
1,E[K2],E[K2],0.03
2,E[K1^2],E[K1^2],0.006
3,E[K2^2],E[K2^2],0.0055
4,Var(K1),E(K1^2)- E[K1]^2,0.0056
5,Var(K2),E(K2^2)- E[K2]^2,0.0046
6,StdDev(K1),sqrt(Var(K1)),0.0748331
7,StdDev(K2),sqrt(Var(K2)),0.0678233
8,E[K1*K2],E[K1*K2],-0.003
9,"Cov(K1, K2)","Cov(K1, K2)",-0.0036


**if std(K1)>std(K2)


# HW8 question 1

In [ ]:
# w1, w2, ...., wm
weights = [0.3, 0.1, 0.6]
means = [0.08, 0.1, 0.06]
stdDevs = [0.15, 0.05, 0.12]

# each entry represent correlation of (i, j) ans stored in the form [(i, j), value]
correlation = [
    [[1, 2], 0.3],
    [[2, 3], 0],
    [[1, 3], -0.2],
]


mu_V = meanV(weights, means) 
cMatrix = makeCMatrix(correlation, stdDevs)
wCwT = find_wCwT(weights, cMatrix)
np.set_printoptions(suppress=True)
print("Expected value of portfolio is:", mu_V)
print("Correlation matrix is:")
print(np.array(cMatrix))
print("w*C*wT is: ", wCwT)
print("stdDev of portfolio is", math.sqrt(wCwT))
print("\n", "*"* 20, "\n")
cInverse = np.linalg.inv(cMatrix)
print("here is the inverse of C:")
print(cInverse)
m = np.array(means)
u = np.array([1 for _ in means])
print("m is:", m)
print("u is:", u)
print("calculating M")
Mmatrix = makeM(m, cInverse, u)
print("here is M:")
print(Mmatrix)
print("here is M inverse:")
MmatrixInv = np.linalg.inv(Mmatrix)
print(MmatrixInv)

# HW 7 Part 2:

In [ ]:
m = [0.2, 0.13, 0.17]
stdDevs = [0.25, 0.28, 0.20]
correlation= [
    [[1,2], 0.25],
    [[2,3], 0],
    [[3,1], 0.15]]

find_W_MVP(m, stdDevs, correlation)

# HW 8, Problem 3:

In [3]:
m = [0.2, 0.1, 0.2]
stdDevs = [0.25, 0.2, 0.25]
correlation= [
    [[1,2], 0.25],
    [[2,3], 0.05],
    [[3,1], 0.15]]

FM.find_W_MVP(m, stdDevs, correlation)

m is: [0.2 0.1 0.2] 
u is: [1 1 1]

C matrix is:
[[0.0625   0.0125   0.009375]
 [0.0125   0.04     0.0025  ]
 [0.009375 0.0025   0.0625  ]]

here is the inverse of C:
 [[17.41882674 -5.29331514 -2.40109141]
 [-5.29331514 26.67121419 -0.2728513 ]
 [-2.40109141 -0.2728513  16.37107776]]

calculating M:
Intermediate Calculations:
M*C^-1 =  [2.47421555 1.55388813 2.76671214]
u*C^-1 =  [ 9.72442019 21.10504775 13.69713506]

here is M:
[[ 1.20357435  6.79481583]
 [ 6.79481583 44.526603  ]]

here is M inverse:
[[ 5.99963235 -0.91555147]
 [-0.91555147  0.16217279]]

The weight in minimum variance portfolio is:
[0.21839574 0.47398738 0.30761689]

The Expected return is:
0.15260126233225074

Variance is:
intermediate calculation: w*C =  [0.02245848 0.02245848 0.02245848]
wCwT = Varinace = 
0.022458483975733808

The Std.Dev of the portfolio is:
0.1498615493571777


# HW 9 Problem 1

In [ ]:
mu = [0.2, 0.15, 0.17]
stdevs = [0.25, 0.28, 0.2]
corr = [
    [[1, 2],0.25],
    [[2,3], 0.0],
    [[1, 3], 0.15]
    ]
muV = 0.2

x = portfolio_MVP(mu, stdevs, corr)
#x.W_MVP_Given_Ev(muV)

# HW 9 Problem 2:

In [ ]:
mu = [0.16, 0.13, 0.13]
stdevs = [0.25, 0.28, 0.2]
corr = [
    [[1, 2],0.3],
    [[2,3], 0.0],
    [[1, 3], 0.15]
    ]

x = portfolio_MVP(mu, stdevs, corr)
x.calculate_ab()
x.calculate_MVL_slope(2, 3)

# HW 9 part 3:

In [7]:
mu = [0.08, 0.1, 0.06]
stdevs = [0.15, 0.05, 0.12]
corr = [
    [[1, 2],0.3],
    [[2,3], 0.0],
    [[1, 3], -0.2]
    ]
risk_free_rate = 0.05
x = FM.portfolio_MVP(mu, stdevs, corr)
x.market_portfolio(risk_free_rate)


m is: [0.08 0.1  0.06] 
u is: [1 1 1]

C matrix is:
[[ 0.0225   0.00225 -0.0036 ]
 [ 0.00225  0.0025   0.     ]
 [-0.0036   0.       0.0144 ]]

here is the inverse of C:
 [[ 51.08556833 -45.97701149  12.77139208]
 [-45.97701149 441.37931034 -11.49425287]
 [ 12.77139208 -11.49425287  72.63729246]]

calculating M:
Intermediate Calculations:
M*C^-1 =  [ 0.25542784 39.77011494  4.23052363]
u*C^-1 =  [ 17.87994891 383.90804598  73.91443167]

here is M:
[[  4.25127714  44.25606641]
 [ 44.25606641 475.70242656]]

here is M inverse:
[[ 7.46276714 -0.69428428]
 [-0.69428428  0.06669357]]

The weight in minimum variance portfolio is:
[0.03758642 0.80703403 0.15537956]

The Expected return of the MVP is:
0.0930330894690919

Variance is:
intermediate calculation: w*C =  [0.00210215 0.00210215 0.00210215]
wCwT = Varinace = 
0.0021021545070138946

The Std.Dev of the portfolio is:
0.045849258521964066
********************

weights in a market portfolio = ((m-R*u) * C^-1)/((m-R*u)* C^-1 * uT)
intermed